# Setup

In [1]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader
import yaml
import os

In [2]:
with open('config.yaml', 'r') as config_file:
    config = yaml.safe_load(config_file)
os.environ['OPENAI_API_KEY'] = config['OPENAI_API_KEY']

In [3]:
openai = ChatOpenAI(model_name='gpt-4o', temperature=0)

# SequentialChain

In [5]:
# Carregar os documentos
loader = TextLoader('base_conhecimento_britadeira.txt')
documents = loader.load()
# Carregar histórico de conversas 
historico_conversas = """Cliente: Minha britadeira não liga. Chatbot: Você já verificou 
                         se a bateria está carregada e conectada corretamente?"""
# Pergunta do cliente
pergunta = "Minha britadeira não liga. Eu já veriquei e a bateria está carregada e conectada corretamente"

In [6]:
inputs = {
    "context": "\n".join(doc.page_content for doc in documents),
    "question": pergunta,
    "historico": historico_conversas
}

In [7]:
prompt_base_conhecimento = PromptTemplate(
    input_variables=["context", "question"],
    template="""Use o seguinte contexto para responder à pergunta. 
    Responda apenas com base nas informações fornecidas.
    Não forneceça instruções de procedimento já realizados.
    Não utilize informações externas ao contexto:
    Contexto: {context}
    Pergunta: {question}"""
)
prompt_historico_conversas = PromptTemplate(
    input_variables=["historico", "question"],
    template="""Use o histórico de conversas para responder à pergunta. 
    Responda apenas com base nas informações fornecidas. 
    Não forneceça instruções de procedimento já realizados.
    Não utilize informações externas ao contexto:
    Histórico: {historico}
    Pergunta: {question}"""
)
prompt_final = PromptTemplate(
    input_variables=["resposta_base_conhecimento", "resposta_historico_conversas"],
    template="""Combine as seguintes respostas para gerar uma resposta final,
    mas não forneça instruções de procedimentos já realizados:
    Resposta da base de conhecimento: {resposta_base_conhecimento}
    Resposta do histórico de conversas: {resposta_historico_conversas}"""
)

In [8]:
print(prompt_base_conhecimento)

input_variables=['context', 'question'] template='Use o seguinte contexto para responder à pergunta. \n    Responda apenas com base nas informações fornecidas.\n    Não forneceça instruções de procedimento já realizados.\n    Não utilize informações externas ao contexto:\n    Contexto: {context}\n    Pergunta: {question}'


In [9]:
# Definir as cadeias  
chain_base_conhecimento = prompt_base_conhecimento | openai
chain_historico_conversas = prompt_historico_conversas | openai
chain_final = prompt_final | openai

In [10]:
print(chain_base_conhecimento)

first=PromptTemplate(input_variables=['context', 'question'], template='Use o seguinte contexto para responder à pergunta. \n    Responda apenas com base nas informações fornecidas.\n    Não forneceça instruções de procedimento já realizados.\n    Não utilize informações externas ao contexto:\n    Contexto: {context}\n    Pergunta: {question}') last=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f9f7d7ec9d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f9f7d57c2d0>, model_name='gpt-4o', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')


In [11]:
print(chain_historico_conversas)

first=PromptTemplate(input_variables=['historico', 'question'], template='Use o histórico de conversas para responder à pergunta. \n    Responda apenas com base nas informações fornecidas. \n    Não forneceça instruções de procedimento já realizados.\n    Não utilize informações externas ao contexto:\n    Histórico: {historico}\n    Pergunta: {question}') last=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f9f7d7ec9d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f9f7d57c2d0>, model_name='gpt-4o', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')


In [12]:
# Passando dados e executando
resultado_base_conhecimento = chain_base_conhecimento.invoke({"context": inputs["context"], "question": inputs["question"]})
resultado_historico_conversas = chain_historico_conversas.invoke({"historico": inputs["historico"], "question": inputs["question"]})
resultado_final = chain_final.invoke({"resposta_base_conhecimento": resultado_base_conhecimento, 
                                      "resposta_historico_conversas": resultado_historico_conversas})

In [13]:
print("Resultado Base de Conhecimento:\n", resultado_base_conhecimento)
print("----")
print("Resultado Histórico de Conversas:\n", resultado_historico_conversas)

Resultado Base de Conhecimento:
 content='Aguarde 30 segundos após conectar a bateria para que o sistema inicialize. Verifique se o interruptor liga/desliga está na posição correta. Se o problema persistir, contate o suporte técnico pelo 0800 555 5555.' response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 681, 'total_tokens': 733, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a288987b44', 'finish_reason': 'stop', 'logprobs': None} id='run-e98007cf-6f5f-49f5-a91b-97339ad47c9e-0' usage_metadata={'input_tokens': 681, 'output_tokens': 52, 'total_tokens': 733}
----
Resultado Histórico de Conversas:
 content='Você já verificou se há algum problema com o interruptor de ligar/desligar ou se há algum dano visível nos cabos ou conectores?' response_metada

In [14]:
print(resultado_final.content)

Aguarde 30 segundos após conectar a bateria para que o sistema inicialize. Verifique se o interruptor liga/desliga está na posição correta e se há algum problema com ele ou danos visíveis nos cabos ou conectores. Se o problema persistir, contate o suporte técnico pelo 0800 555 5555.


In [15]:
print(resultado_final)

content='Aguarde 30 segundos após conectar a bateria para que o sistema inicialize. Verifique se o interruptor liga/desliga está na posição correta e se há algum problema com ele ou danos visíveis nos cabos ou conectores. Se o problema persistir, contate o suporte técnico pelo 0800 555 5555.' response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 493, 'total_tokens': 562, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-2921e5be-5f96-4c43-adc9-0baca7a7215b-0' usage_metadata={'input_tokens': 493, 'output_tokens': 69, 'total_tokens': 562}
